In [2]:
#This script loads a 3D ResNet model, trains it on the mitosis training data and evaluates on test data

#To be run in workenvsam
import torch
import os
os.chdir('3D-ResNets-PyTorch')

print('Available GPU: ',torch.cuda.get_device_name(0))

#!git clone https://github.com/kenshohara/3D-ResNets-PyTorch.git

Available GPU:  NVIDIA GeForce RTX 3090


In [21]:
#To prepare the data
#This is equivalent to running mitoses_json.py with the inputs as strings from the command window. All to produce a .json file
#from csv files. The csv files have been created using a Colab Notebook from the original images

!python -m util_scripts.mitoses_json ../data/csvData 2 ../data/jpg_videos jpg ../data/mitosesData2.json 0.43


Namespace(dir_path=WindowsPath('../data/csvData'), n_classes=2, video_path=WindowsPath('../data/jpg_videos'), video_type='jpg', dst_path=WindowsPath('../data/mitosesData2.json'), split_val=0.43)
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG
FLAG


In [26]:
#Inspection of the resnet 3D model architecture
model = resnet.generate_model(model_depth=18,n_classes=2)
print(model)

ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [14]:
#>>>>>>>>>>>>>>>>>>>>TRAINING<<<<<<<<<<<<<<<<<<<<<
#Example for resnet50 with bs 10, 500 epochs, pretraining with the 700 classes dataset.

!python main.py --root_path ../data --video_path jpg_videos --annotation_path mitosesData.json --result_path resultsResNet50/epoch500lr0.001bs10pretYes --dataset mitoses 
--model resnet --model_depth 50 --n_classes 2 --batch_size 10 --n_threads 4 --checkpoint 5 --sample_duration 12 --mean_dataset 0.5 --n_epochs 500 --n_pretrain_classes 700

^C


In [14]:
#Evaluation on "testing" data 1 and 2 for 500 epochs training depth 50, lr = 0.001 no pretraining
!python main.py --root_path ../data --video_path jpg_videos --annotation_path mitosesData.json --result_path resultsResNet50/epoch500lr0.001bs10pretNo\
--dataset mitoses --resume_path resultsResNet50/epoch500lr0.001bs10pretNo/save_500.pth --model_depth 50 --n_classes 2 --n_threads 4 --no_train --no_val \
--inference --inference_subset test --output_topk 5 --inference_batch_size 1 

!python -m util_scripts.eval_accuracy ../data/mitosesData.json ../data/resultsResNet50/epoch500lr0.001bs10pretNo/test.json --subset testing -k 1 --ignore --save --otherMeasures

Namespace(root_path=WindowsPath('../data'), video_path=WindowsPath('../data/jpg_videos'), annotation_path=WindowsPath('../data/mitosesData.json'), result_path=WindowsPath('../data/resultsResNet50/epoch500lr0.001bs10pretNo'), dataset='mitoses', n_classes=2, n_pretrain_classes=0, pretrain_path=None, ft_begin_module='', sample_size=112, sample_duration=16, sample_t_stride=1, train_crop='random', train_crop_min_scale=0.25, train_crop_min_ratio=0.75, no_hflip=False, colorjitter=False, train_t_crop='random', learning_rate=0.1, momentum=0.9, dampening=0.0, weight_decay=0.001, mean_dataset='kinetics', no_mean_norm=False, no_std_norm=False, value_scale=1, nesterov=False, optimizer='sgd', lr_scheduler='multistep', multistep_milestones=[50, 100, 150], overwrite_milestones=False, plateau_patience=10, batch_size=128, inference_batch_size=1, batchnorm_sync=False, n_epochs=200, n_val_samples=3, resume_path=WindowsPath('../data/resultsResNet50/epoch500lr0.001bs10pretNo/save_500.pth'), no_train=True, n

In [15]:
#Evaluation on "testing" data 1 and 2 for 500 epochs training depth 50, lr = 0.001 pretrained
!python main.py --root_path ../data --video_path jpg_videos --annotation_path mitosesData.json --result_path resultsResNet50/epoch500lr0.001bs10pretYes\
--dataset mitoses --resume_path resultsResNet50/epoch500lr0.001bs10pretYes/save_500.pth --model_depth 50 --n_classes 2 --n_threads 4 --no_train --no_val \
--inference --inference_subset test --output_topk 5 --inference_batch_size 1 

!python -m util_scripts.eval_accuracy ../data/mitosesData.json ../data/resultsResNet50/epoch500lr0.001bs10pretYes/test.json --subset testing -k 1 --ignore --save --otherMeasures

Namespace(root_path=WindowsPath('../data'), video_path=WindowsPath('../data/jpg_videos'), annotation_path=WindowsPath('../data/mitosesData.json'), result_path=WindowsPath('../data/resultsResNet50/epoch500lr0.001bs10pretYes'), dataset='mitoses', n_classes=2, n_pretrain_classes=0, pretrain_path=None, ft_begin_module='', sample_size=112, sample_duration=16, sample_t_stride=1, train_crop='random', train_crop_min_scale=0.25, train_crop_min_ratio=0.75, no_hflip=False, colorjitter=False, train_t_crop='random', learning_rate=0.1, momentum=0.9, dampening=0.0, weight_decay=0.001, mean_dataset='kinetics', no_mean_norm=False, no_std_norm=False, value_scale=1, nesterov=False, optimizer='sgd', lr_scheduler='multistep', multistep_milestones=[50, 100, 150], overwrite_milestones=False, plateau_patience=10, batch_size=128, inference_batch_size=1, batchnorm_sync=False, n_epochs=200, n_val_samples=3, resume_path=WindowsPath('../data/resultsResNet50/epoch500lr0.001bs10pretYes/save_500.pth'), no_train=True,

In [1]:
#Evaluation on "testing" data 1 and 2 for 500 epochs training depth 50, lr = 0.01  bs20 pretrained
!python main.py --root_path ../data --video_path jpg_videos --annotation_path mitosesData.json --result_path resultsResNet50/epoch500lr0.01bs20pretYes\
--dataset mitoses --resume_path resultsResNet50/epoch500lr0.01bs20pretYes/save_500.pth --model_depth 50 --n_classes 2 --n_threads 4 --no_train --no_val \
--inference --inference_subset test --output_topk 5 --inference_batch_size 1 

!python -m util_scripts.eval_accuracy ../data/mitosesData.json ../data/resultsResNet50/epoch500lr0.01bs20pretYes/test.json --subset testing -k 1 --ignore --save --otherMeasures

Namespace(root_path=WindowsPath('../data'), video_path=WindowsPath('../data/jpg_videos'), annotation_path=WindowsPath('../data/mitosesData.json'), result_path=WindowsPath('../data/resultsResNet50/epoch500lr0.01bs20pretYes'), dataset='mitoses', n_classes=2, n_pretrain_classes=0, pretrain_path=None, ft_begin_module='', sample_size=112, sample_duration=16, sample_t_stride=1, train_crop='random', train_crop_min_scale=0.25, train_crop_min_ratio=0.75, no_hflip=False, colorjitter=False, train_t_crop='random', learning_rate=0.1, momentum=0.9, dampening=0.0, weight_decay=0.001, mean_dataset='kinetics', no_mean_norm=False, no_std_norm=False, value_scale=1, nesterov=False, optimizer='sgd', lr_scheduler='multistep', multistep_milestones=[50, 100, 150], overwrite_milestones=False, plateau_patience=10, batch_size=128, inference_batch_size=1, batchnorm_sync=False, n_epochs=200, n_val_samples=3, resume_path=WindowsPath('../data/resultsResNet50/epoch500lr0.01bs20pretYes/save_500.pth'), no_train=True, n

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\workenvsam\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\Anaconda3\envs\workenvsam\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "E:\MitosisClassifierToCompare\3D-ResNets-PyTorch\util_scripts\eval_accuracy.py", line 128, in <module>
    accuracy,precision,recall,f1 = evaluate(args.ground_truth_path, args.result_path, args.subset,
  File "E:\MitosisClassifierToCompare\3D-ResNets-PyTorch\util_scripts\eval_accuracy.py", line 91, in evaluate
    precision = truePos / (truePos + falsePos)
ZeroDivisionError: division by zero
